# 统计与大数据分析软件第六次作业

## 第一题

如果有`dataframe df = pd.DataFrame({'X': [7, 2, 0, 3, 4, 2, 5, 0, 3, 4]})`, 

那么对于列X的每一个值，计算相对于离其最近的0的相对位置（如果前面没有0，则计算从头开始的位置），


这样就会得到`[1, 2, 0, 1, 2, 3, 4, 0, 1, 2]`，我们可以把它作为df的新的列'Y'.

问题：让用户输入任意一列整数，构造新的`dataframe`，以此列整数为列X，并按照上述方法计算新的列Y。

首先我们导入`Pandas`的包：

In [2]:
import pandas as pd
import numpy as np

设计一段函数，实现我们的分割功能：

In [3]:
def compute_relative_position_pandas(series):
    # 计算相对位置(使用numpy的函数标记索引为0的元素)
    zero_mask = series.eq(0)
    zero_indices = np.where(zero_mask)[0]
    
    # 初始化距离数组，并设置第一个0的索引（初始情况）
    distances = np.zeros_like(series)
    last_zero = -1
    
    # 遍历序列，计算每个元素到最近的0的距离
    for i in range(len(series)):
        if i in zero_indices:
            last_zero = i # 更新最近的0的索引
        distances[i] = i - last_zero    # 计算距离
    
    return pd.Series(distances, index=series.index)


主程序如下：

In [4]:
for i in range(2):
    user_input = input("请输入一列整数，用空格分隔: ")
    numbers = [int(num) for num in user_input.split()]
    df = pd.DataFrame({'X': numbers})
    df['Y'] = compute_relative_position_pandas(df['X'])
    print(df)

   X  Y
0  2  1
1  4  2
2  5  3
3  0  0
4  4  1
5  3  2
6  8  3
7  0  0
8  1  1
9  6  2
   X  Y
0  3  1
1  7  2
2  8  3
3  2  4
4  3  5
5  1  6


## 第二题

滑动平均：给定`dataframe`包含两列，一列为`group`，一列为`value`，

例如`df = pd.DataFrame({'group': list('aabbabbbabab')`,`'value': [1, 2, 3, np.nan, 2, 3, np.nan, 1, 7, 3, np.nan, 8]})`

请根据`group`的进行滑动平均，自己定义窗口尺寸，如遇到`nan`请忽略（注意不是作为0处理，是忽略）。

例如上边的例子结果应为（窗口长度为3）：（此处视为6位，省略至小数点后4位）

| 索引 | 0 | 1 | 2 | 3 | 4 | 5 | 6 | 7 | 8 | 9 | 10 | 11 |
| :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: | :---: |
| 值 | 1.0000 | 1.5000 | 3.0000 | 3.0000 | 1.6667 | 3.0000 | 3.0000 | 2.0000 | 3.6667 | 2.0000 | 4.5000 | 4.0000 |

首先，导入相应的包：

In [22]:
import pandas as pd
import numpy as np

设计我们的主体函数：

In [23]:
def group_rolling_mean(df, window=3):
    def rolling_avg(s):
        return s.rolling(window, min_periods=1).mean()
    
    df['smoothed'] = df.groupby('group')['value'].transform(rolling_avg)
    return df

新建一个pandas表，如下：

In [24]:
df = pd.DataFrame({'group': list('aabbabbbabab'),
                   'value': [1, 2, 3, np.nan, 2, 3, np.nan, 1, 7, 3, np.nan, 8]})
df

,group,value
0,a,1.0
1,a,2.0
2,b,3.0
3,b,NaN
4,a,2.0
5,b,3.0
6,b,NaN
7,b,1.0
8,a,7.0
9,b,3.0


调用我们的函数得出计算结果：

In [30]:
df_result = group_rolling_mean(df, window=3)
print(df_result['smoothed'].round(6).tolist())

output_df = pd.DataFrame({
    '索引': df.index,
    '值': df_result['smoothed'].round(6)
})

# 设置索引列
output_df.set_index('索引', inplace=True)

# 输出
print(output_df)

[1.0, 1.5, 3.0, 3.0, 1.666667, 3.0, 3.0, 2.0, 3.666667, 2.0, 4.5, 4.0]
           值
索引          
0   1.000000
1   1.500000
2   3.000000
3   3.000000
4   1.666667
5   3.000000
6   3.000000
7   2.000000
8   3.666667
9   2.000000
10  4.500000
11  4.000000


我们还可以调整窗口大小：

In [31]:
window = int(input("请输入窗口大小: "))
df0_result = group_rolling_mean(df, window)
print("窗口大小：", window)
print(df0_result['smoothed'].round(6).tolist())

output_df0 = pd.DataFrame({
    '索引': df.index,
    '值': df0_result['smoothed'].round(6)
})

# 设置索引列
output_df0.set_index('索引', inplace=True)

# 输出
print(output_df0)

窗口大小： 4
[1.0, 1.5, 3.0, 3.0, 1.666667, 3.0, 3.0, 2.0, 3.0, 2.333333, 3.666667, 4.0]
           值
索引          
0   1.000000
1   1.500000
2   3.000000
3   3.000000
4   1.666667
5   3.000000
6   3.000000
7   2.000000
8   3.000000
9   2.333333
10  3.666667
11  4.000000


## 第三题

扫雷游戏。创建一个dataframe来模拟扫雷游戏：

(1) 创建一个扫雷游戏，即创建一个dataframe，包含两列X和Y，例如5*4的扫雷游戏，

这个dataframe分别记录5*4个格子的坐标，这个dataframe为两列，20行，它的一部分如下：

|   | x | y |
| :---: | :---: | :---: |
| 0 | 0 | 0 |
| 1 | 0 | 1 |
| 2 | 0 | 2 |

（2）增加一列，此列为格子是否为雷，如果是的话值为1，否则为0. （使用随机函数，每个位置为雷的概率为0.4）

（3）再增加一列adjacent，此列记录当前格子相邻的格子的雷的数目，如果当前格子为雷，则值为NaN。

（4）创建一个新的5行4列的dataframe，其中值为(3)中计算得到的雷的数目。

首先，我们需要导入必要的库：

In [11]:
import pandas as pd
import numpy as np

对于第一题，我们先建立一个`dataframe`:

1. 创建 5×4 的坐标dataframe

In [13]:
user_input = input("请输入两个整数，用空格分隔: ")
X, Y = [int(num) for num in user_input.split()]

# 构造所有 (x, y) 坐标，其中 x 从 0 到 X-1，y 从 0 到 Y-1
coords = [(x, y) for x in range(X) for y in range(Y)]
df = pd.DataFrame(coords, columns=['x', 'y'])
print(f"初始坐标：({X}, {Y})")
print(df.head()) # 为了美观我们转置输出

初始坐标：(5, 4)
   x  y
0  0  0
1  0  1
2  0  2
3  0  3
4  1  0


2. 增加一列表示是否为雷（1为雷，0为非雷），每个格子成为雷的概率为 0.4

In [14]:
np.random.seed(0)  # 固定随机种子，便于结果复现
df['bomb'] = np.random.choice([1, 0], size=len(df), p=[0.4, 0.6])
print("\n增加雷标记后：")
print(df.head(10))


增加雷标记后：
   x  y  bomb
0  0  0     0
1  0  1     0
2  0  2     0
3  0  3     0
4  1  0     0
5  1  1     0
6  1  2     0
7  1  3     0
8  2  0     0
9  2  1     1


3. 增加一列记录相邻格子中雷的个数，若当前格子本身为雷，则该值为 NaN

In [15]:
# 定义一个函数来计算相邻格子中的雷数
def count_adjacent_bombs(row, df):
    # 如果本身为雷，则返回NaN
    if row['bomb'] == 1:
        return np.nan
    # x,y坐标
    x, y = row['x'], row['y']
    # 定义相邻坐标的范围（注意边界）
    adjacent = df[
        (df['x'] >= x-1) & (df['x'] <= x+1) &
        (df['y'] >= y-1) & (df['y'] <= y+1) &
        ~((df['x'] == x) & (df['y'] == y))  # 排除本身
    ]
    # 返回相邻格子中雷的个数
    return adjacent['bomb'].sum()

In [16]:
df['adjacent'] = df.apply(lambda row: count_adjacent_bombs(row, df), axis=1)
print("\n增加相邻雷数后：")
print(df.head(10))


增加相邻雷数后：
   x  y  bomb  adjacent
0  0  0     0       0.0
1  0  1     0       0.0
2  0  2     0       0.0
3  0  3     0       0.0
4  1  0     0       1.0
5  1  1     0       1.0
6  1  2     0       1.0
7  1  3     0       0.0
8  2  0     0       1.0
9  2  1     1       NaN


4. 创建一个新的 5×4 的dataframe，其中的值为 (3) 中计算得到的相邻雷数

In [17]:
# 使用 pivot 将原 dataframe 重构为二维矩阵
grid = df.pivot(index='x', columns='y', values='adjacent')
print(f"\n{X}行{Y}列的相邻雷数矩阵：")
print(grid)


5行4列的相邻雷数矩阵：
y    0    1    2    3
x                    
0  0.0  0.0  0.0  0.0
1  1.0  1.0  1.0  0.0
2  1.0  NaN  3.0  2.0
3  2.0  3.0  NaN  NaN
4  NaN  2.0  2.0  2.0
